In [1]:
from src import *

In [2]:
num_points = 10000
df = pd.DataFrame()
num_treatments = 4

treat_cols = []
for i in range(num_treatments):
    treat_name = 'z'+str(i)
    df[treat_name] = (2 * (np.random.randint(0, 2, num_points) - 0.5)).astype(int)
    treat_cols.append(treat_name)

num_covs = 3
cov_cols = []
for i in range(num_covs):
    cov_name = 'x'+str(i)
    df[cov_name] = np.random.randint(0, 2, num_points)
    cov_cols.append(cov_name)
    
df['y'] = np.random.randint(0, 2, num_points)

In [4]:
fw = FactorialWeights()
effects = ['z0']
fw.heterogeneous_treat_model(df, treat_cols, cov_cols, effects)

Variable((np.int32(10000),), var222)

In [ ]:
df